<a href="https://colab.research.google.com/github/ValentinCord/HandsOnAI_2/blob/main/NLP_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!/opt/bin/nvidia-smi
!rm -rf sample_data

!pip3 install transformers
!pip3 install datasets
!pip install sentencepiece

Mon Dec 19 20:03:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# basics 
import os
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn import metrics

# transformers 
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer
from transformers import CamembertModel, CamembertTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# plot 
import matplotlib.pyplot as plt 
import seaborn as sns 

# torch 
import torch
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

# nltk 
import re
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 10
VALID_BATCH_SIZE = 10
EPOCHS = 1
LEARNING_RATE = 1e-05

LEN_TEXT = 300
OVERLAP = 50

DONNEE_AJOUTEES = 200

TRANSFORMER_NAME = "cmarkea/distilcamembert-base"
TRAIN_SIZE = 0.8

In [5]:
train_path = '/content/drive/MyDrive/HandOnAI_2_NLP/fake_train.csv'
added_path = '/content/drive/MyDrive/HandOnAI_2_NLP/added_train.csv'
test_path = '/content/drive/MyDrive/HandOnAI_2_NLP/fake_test.csv'

df = pd.read_csv(train_path)
df_added = pd.read_csv(added_path)
df_test = pd.read_csv(test_path)

# suppression des colonnes inutiles 
df = df.drop(['Unnamed: 0', 'target_name'], axis = 1)
df_added.rename(columns = {'french':'data'}, inplace = True)
df_added = df_added.drop(['Unnamed: 0'], axis = 1)
df_test = df_test.drop(['Unnamed: 0', 'target_name'], axis = 1)

df = df.append(df_added[:DONNEE_AJOUTEES], ignore_index=True)

In [6]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('french'))

def clean_text(text):
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    #text = BAD_SYMBOLS_RE.sub('', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

In [7]:
df['data'] = df['data'].apply(clean_text)
df_test['data'] = df_test['data'].apply(clean_text)

df = df.drop(df.index[1136])
df = df.reset_index()

In [8]:
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

df_train, df_val = train_test_split(df, test_size=0.2, shuffle=True)

MAX_SEQ_LEN = 5000

X_train = df_train.data.tolist()
X_val = df_val.data.tolist()
X_test = df_test.data.tolist()

y_train = df_train.label.tolist()
y_val = df_val.label.tolist()
y_test = df_test.label.tolist()

train_text_vec = [text for text in X_train]
val_text_vec = [text for text in X_val]
test_text_vec = [text for text in X_test]

# tokenize the sentences
tokenizer = Tokenizer(lower=False)
tokenizer.fit_on_texts(train_text_vec)

train_text_vec = tokenizer.texts_to_sequences(train_text_vec)
val_text_vec = tokenizer.texts_to_sequences(val_text_vec)
test_text_vec = tokenizer.texts_to_sequences(test_text_vec)

# pad the sequences
train_text_vec = pad_sequences(train_text_vec, maxlen=MAX_SEQ_LEN)
val_text_vec = pad_sequences(val_text_vec, maxlen=MAX_SEQ_LEN)
test_text_vec = pad_sequences(test_text_vec, maxlen=MAX_SEQ_LEN)


In [9]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# One Hot Encode Y values:
encoder = LabelEncoder()

y_train = encoder.fit_transform(df_train['label'].values)
y_train = to_categorical(y_train) 

y_val = encoder.fit_transform(df_val['label'].values)
y_val = to_categorical(y_val) 

y_test = encoder.fit_transform(df_test['label'].values)
y_test = to_categorical(y_test) 


In [10]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam 

model = Sequential()
model.add(Embedding(input_dim = (len(tokenizer.word_counts) + 1), output_dim = 128, input_length = MAX_SEQ_LEN))
model.add(LSTM(128))
model.add(Dense(64, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [15]:
batch_size = 64
num_epochs = 10
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_text_vec, y_train, validation_data=(val_text_vec, y_val), batch_size=batch_size, epochs=num_epochs)

Epoch 1/10
21/21 [==============================] - 8s 277ms/step - loss: 0.0900 - accuracy: 0.9834 - val_loss: 0.2292 - val_accuracy: 0.9247
Epoch 2/10
21/21 [==============================] - 5s 238ms/step - loss: 0.0374 - accuracy: 0.9932 - val_loss: 0.2391 - val_accuracy: 0.9398
Epoch 3/10
21/21 [==============================] - 6s 278ms/step - loss: 0.0539 - accuracy: 0.9894 - val_loss: 0.2313 - val_accuracy: 0.9187
Epoch 4/10
21/21 [==============================] - 5s 244ms/step - loss: 0.0211 - accuracy: 0.9962 - val_loss: 0.5055 - val_accuracy: 0.8765
Epoch 5/10
21/21 [==============================] - 5s 245ms/step - loss: 0.0409 - accuracy: 0.9925 - val_loss: 0.5078 - val_accuracy: 0.8012
Epoch 6/10
21/21 [==============================] - 5s 247ms/step - loss: 0.0113 - accuracy: 0.9977 - val_loss: 0.4341 - val_accuracy: 0.8193
Epoch 7/10
21/21 [==============================] - 5s 245ms/step - loss: 7.5723e-04 - accuracy: 1.0000 - val_loss: 0.6502 - val_accuracy: 0.8554
Ep

In [16]:
test_scores = model.evaluate(test_text_vec, y_test, verbose=1)

print("test scores:", test_scores)

16/16 [==============================] - 1s 81ms/step - loss: 0.4816 - accuracy: 0.8683
test scores: [0.481625497341156, 0.8683127760887146]
